# How to use Xenomapper2

This document is a [jupyter](http://jupyter.org) notebook that illustrates some of the ways to use xenomapper2.
If you are viewing this on github it will have the output of each of the commands shown, or alternatively you can download the .ipynb file and run the examples on your own system.  All commands are executed on the command line, and the `%%bash` should be ingnored if you are following along at the command line (the `%%bash` just tells jupyter to execute the cells code with bash when this document is run as a notebook).

Xenomapper2 is a tool for post processing reads that come from a biological source where two genomes are present and the reads have been aligned to both of these reference genomes.  Xenomapper2 compares the primary mapping scores between the two genomes, and if available the suboptimal alignment mapping score to assess uniqueness within the genome, to determine the specificity of the read and template.  Unlike compound genome mapping approaches this is capable of resolving species specific multimapping reads.

Before using Xenomapper2 you must have mapped your reads to both genomes of interest.  Xenomapper2 can be used with most common aligners including [bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml), [HISAT2](https://ccb.jhu.edu/software/hisat2/), and [BWA-MEM](https://github.com/lh3/bwa).  The dataset that we will use is from [Rossello et al](http://dx.doi.org/10.1371/journal.pone.0074432) and is a publicly available cell line data set.  Although this data set is older we use it as it does not require any restricted access.  (Note the downloads for running this example total 700MB for the data and 7GB for the references)

# Installation

This example requires bowtie2, sratoolkit, and samtools.  If you do not already have these installed the easiest way to install these requirements on linux or MacOS is to use [homebrew](http://brew.sh) or [linuxbrew](http://linuxbrew.sh)

In [ ]:
%%bash
brew install bowtie2
brew install sratoolkit
brew install samtools

In [8]:
%%bash
brew info bowtie2 samtools sratoolkit

bowtie2: stable 2.3.5.1 (bottled)
Fast and sensitive gapped read aligner
https://bowtie-bio.sourceforge.io/bowtie2/
/usr/local/Cellar/bowtie2/2.3.5.1 (79 files, 24.1MB) *
  Poured from bottle on 2020-02-18 at 14:28:30
From: https://github.com/Homebrew/homebrew-core/blob/master/Formula/bowtie2.rb
==> Dependencies
Required: tbb
==> Analytics
install: 156 (30 days), 459 (90 days), 2,760 (365 days)
install-on-request: 119 (30 days), 365 (90 days), 2,027 (365 days)
build-error: 0 (30 days)

samtools: stable 1.10 (bottled)
Tools for manipulating next-generation sequencing data
https://www.htslib.org/
/usr/local/Cellar/samtools/1.10 (58 files, 959KB) *
  Poured from bottle on 2020-02-18 at 14:33:20
From: https://github.com/Homebrew/homebrew-core/blob/master/Formula/samtools.rb
==> Dependencies
Required: htslib
==> Analytics
install: 528 (30 days), 1,868 (90 days), 6,096 (365 days)
install-on-request: 474 (30 days), 1,657 (90 days), 5,586 (365 days)
build-error: 0 (30 days)

sratoolkit: stable

In [3]:
%%bash
#pip3 install --upgrade xenomapper2
pip3 install --upgrade ../xenomapper2

Processing /Users/wakefield/PycharmProjects/xenomapper2
  Stored in directory: /private/var/folders/3_/7gn1zhl111q5c1cx1dvr2pt00000gp/T/pip-ephem-wheel-cache-721tyklf/wheels/b2/99/37/17f1c2e8a439770abaa348659b9bbf10e63d6c67db7688beae
Successfully built xenomapper2


# Download data files
The Short Read Archive uses its own file format so we need to use their tool to access the data.  Unfortunately all this is very slow so downloading the full dataset can take a long time.  To speed up the process we will only download the first 1 Million reads.  (Alternatively you could use [Aspera connect](http://www.ncbi.nlm.nih.gov/books/NBK242625/))
You need to preserve the original readnames (--origfmt), and want singleton reads in a separate file (--split-3).

In [9]:
%%bash
fastq-dump -X 1000000 --origfmt --split-3 SRR879369


Read 1000000 spots for SRR879369
Written 1000000 spots for SRR879369


We also need to have both the bowtie2 index for both human and mouse so we will download the prebuilt versions from the bowtie2 web page

In [10]:
%%bash
curl -O ftp://ftp.ncbi.nlm.nih.gov/genomes/archive/old_genbank/Eukaryotes/vertebrates_mammals/Homo_sapiens/GRCh38/seqs_for_alignment_pipelines/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3575M  100 3575M    0     0  7013k      0  0:08:42  0:08:42 --:--:-- 8160k


In [11]:
%%bash
curl -O ftp://ftp.ccb.jhu.edu/pub/data/bowtie2_indexes/mm10.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3213M  100 3213M    0     0  7195k      0  0:07:37  0:07:37 --:--:-- 8008k


In [12]:
%%bash
mkdir genomes
tar xzf GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index.tar.gz -C genomes/
unzip mm10.zip -d genomes/

Archive:  mm10.zip
  inflating: genomes/mm10.1.bt2      
  inflating: genomes/mm10.2.bt2      
  inflating: genomes/mm10.3.bt2      
  inflating: genomes/mm10.4.bt2      
  inflating: genomes/mm10.rev.1.bt2  
  inflating: genomes/mm10.rev.2.bt2  
  inflating: genomes/make_mm10.sh    


# Mapping the data

When Xenomapper compares the output of the mapping against the two genomes it needs to be able to locate the same read in both genomes while simultaneously traversing the two files.  This is trivial if the reads are maintained in the same order through the mapping process, so choosing our mapping options carefully avoids the need to resort the mapped reads by name. (Sorting works if you need to do this with previously mapped data, just use the same tool (e.g. samtools sort) on both files as lexicial sort order varies between programs).

Most aligners in single thread mode will return the results in the same order that they occur in the fastq input file, and in mutlithreaded mode there is usually an option to preserve the read order.

From the downloads above you should have two fastq files in your working directory and 12 .bt2 bowtie2 index files in the genomes folder.

In [17]:
%%bash
bowtie2 --local --threads 7 --reorder \
         -x genomes/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index \
         -1 SRR879369_1.fastq \
         -2 SRR879369_2.fastq \
         | samtools view -b - > SRR879369.hs.bam


1000000 reads; of these:
  1000000 (100.00%) were paired; of these:
    152190 (15.22%) aligned concordantly 0 times
    607098 (60.71%) aligned concordantly exactly 1 time
    240712 (24.07%) aligned concordantly >1 times
    ----
    152190 pairs aligned concordantly 0 times; of these:
      74655 (49.05%) aligned discordantly 1 time
    ----
    77535 pairs aligned 0 times concordantly or discordantly; of these:
      155070 mates make up the pairs; of these:
        40931 (26.40%) aligned 0 times
        36482 (23.53%) aligned exactly 1 time
        77657 (50.08%) aligned >1 times
97.95% overall alignment rate


In [18]:
%%bash
bowtie2 --local --threads 7 --reorder \
         -x genomes/mm10 \
         -1 SRR879369_1.fastq \
         -2 SRR879369_2.fastq \
         | samtools view -b - > SRR879369.mm.bam



1000000 reads; of these:
  1000000 (100.00%) were paired; of these:
    978336 (97.83%) aligned concordantly 0 times
    7804 (0.78%) aligned concordantly exactly 1 time
    13860 (1.39%) aligned concordantly >1 times
    ----
    978336 pairs aligned concordantly 0 times; of these:
      2787 (0.28%) aligned discordantly 1 time
    ----
    975549 pairs aligned 0 times concordantly or discordantly; of these:
      1951098 mates make up the pairs; of these:
        1839070 (94.26%) aligned 0 times
        51395 (2.63%) aligned exactly 1 time
        60633 (3.11%) aligned >1 times
8.05% overall alignment rate


We can see from the bowtie output that the majority of reads in this sample are human.  Mapping to the human genome is sufficient to identify the concordant reads with paired end at this length; however, we are much more interested in the discordant reads as these will be important cancer structural variations.  Also of note is that the numbers don't add up, there are clearly reads with valid alignments to both genomes that need disambiguating based on the alignment score. 

# Catagorizing reads with Xenomapper2

In [22]:
%%bash
xenomapper2 --primary SRR879369.hs.bam \
            --secondary SRR879369.mm.bam \
            --basename SRR879369


xenomapper2 v2.0a1 --primary=SRR879369.hs.bam --secondary=SRR879369.mm.bam --primary-specific=None --primary-multi=None --secondary-specific=None --secondary-multi=None --unassigned=None --unresolved=None --basename=SRR879369 --min-score=None --zs=False --cigar=False --max=False --conservative=False --version=False

--------------------------------------------------------------------------------

Read Category Summary

|       Category                                     |     Count       |
|:--------------------------------------------------:|:---------------:|
|  primary_multi & primary_multi                     |          58669  |
|  primary_multi & primary_specific                  |          18113  |
|  primary_multi & secondary_multi                   |            136  |
|  primary_multi & secondary_specific                |            319  |
|  primary_multi & unresolved                        |             73  |
|  primary_specific & primary_multi                  |          1

The summary table is markdown formated.  Also note that the summary for paired end data treats forward and reverse orientations of discord as separate categories.  This is because the reverse read is frequently of lower quality that forward read and this information can be useful for tuning quality thresholds.

### Read Category Summary

|       Category                                     |     Count       |
|:--------------------------------------------------:|:---------------:|
|  primary_multi & primary_multi                     |          58669  |
|  primary_multi & primary_specific                  |          18113  |
|  primary_multi & secondary_multi                   |            136  |
|  primary_multi & secondary_specific                |            319  |
|  primary_multi & unresolved                        |             73  |
|  primary_specific & primary_multi                  |          18861  |
|  primary_specific & primary_specific               |         875734  |
|  primary_specific & secondary_multi                |            389  |
|  primary_specific & secondary_specific             |            866  |
|  primary_specific & unassigned                     |           4089  |
|  primary_specific & unresolved                     |            206  |
|  secondary_multi & primary_multi                   |            136  |
|  secondary_multi & primary_specific                |            331  |
|  secondary_multi & secondary_multi                 |            111  |
|  secondary_multi & secondary_specific              |            147  |
|  secondary_multi & unresolved                      |             24  |
|  secondary_specific & primary_multi                |            439  |
|  secondary_specific & primary_specific             |           1015  |
|  secondary_specific & secondary_multi              |            160  |
|  secondary_specific & secondary_specific           |            403  |
|  secondary_specific & unassigned                   |            178  |
|  secondary_specific & unresolved                   |             34  |
|  unassigned & primary_specific                     |           1862  |
|  unassigned & secondary_specific                   |           1245  |
|  unassigned & unassigned                           |          15866  |
|  unassigned & unresolved                           |            123  |
|  unresolved & primary_multi                        |            102  |
|  unresolved & primary_specific                     |            221  |
|  unresolved & secondary_multi                      |             10  |
|  unresolved & secondary_specific                   |             28  |
|  unresolved & unassigned                           |             23  |
|  unresolved & unresolved                           |             87  |


You can also specify addition output files for other categories of read.  For example if we were interested in oncoviruses we could look in the unassigned category, although the more usual contents is poor quality reads and adaptor sequences.

In [30]:
%%bash
samtools view SRR879369_unassigned.bam | head

D81P8DQ1:109:D1CW8ACXX:4:1101:5445:2187	77	*	0	0	*	*	0	0	GATCGGAAGAGCACACGTCTGANCTCCAGTCACCCGTCCCGATCTCNNNTGCCGGCTTCTGCTTGCAAAAAAACAAACGCAGAGTAGAGTGGACATGCTCA	@BCFFFFFHGHHHJIJJEHIJI#1?FHEHCGGJIJHHIJJDA7@FG#######################################################	YT:Z:UP
D81P8DQ1:109:D1CW8ACXX:4:1101:5445:2187	141	*	0	0	*	*	0	0	GGNNNGNGAAAGGGGGGGNGNGNGNGGGGGGGNNNGGGGGGGGGGGNGGGGGGGTCTCTTTAAAAAAAAAAAAAAAAAAACACACAACAACGTACGANTGA	1?###################################################################################################	YT:Z:UP
D81P8DQ1:109:D1CW8ACXX:4:1101:9772:2187	77	*	0	0	*	*	0	0	GATCGGAAGAGCACACGTCTGANCTCCAGTCACCNGTCCCGATCTCNNNTGCCGTCTTCTGCTTTAAAAAAAAANAATAAAGAACAACAAAACGCAGTACA	CCCFFFFFGFGHHJJIJFHIJJ#1@EGGHHIJJJ#08BGGA7@G>F#######################################################	YT:Z:UP
D81P8DQ1:109:D1CW8ACXX:4:1101:9772:2187	141	*	0	0	*	*	0	0	GGNNNGNGGGGGAGGGGGNGNGNGNGAAGGGGNNNGAGNGGTGGGGNGGCGCGCGACTTTNAAAAAAAAAAAAAAAAAATAGCATAAAATACGTAATNGCT	################################